# Data Mining Project 2

https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Archived-05-11-2022-Residential-Sales-Dat/5pge-nu6u/about_data

1. Data Acquisition

In [4]:
import requests
import pandas as pd

In [38]:
pd.set_option('display.max_columns', None)

In [34]:
df = pd.read_csv("mini2/cook_county_train_val.csv")

In [35]:
print(len(df))

204792


In [39]:
print(df.head())

   Unnamed: 0             PIN  Property Class  Neighborhood Code  \
0           0  17294100610000             203                 50   
1           1  13272240180000             202                120   
2           2  25221150230000             202                210   
3           3  10251130030000             203                220   
4           4  31361040550000             202                120   

   Land Square Feet  Town Code  Apartments  Wall Material  Roof Material  \
0            2500.0         76         0.0            2.0            1.0   
1            3780.0         71         0.0            2.0            1.0   
2            4375.0         70         0.0            2.0            1.0   
3            4375.0         17         0.0            3.0            1.0   
4            8400.0         32         0.0            3.0            1.0   

   Basement  Basement Finish  Central Heating  Other Heating  Central Air  \
0       1.0              3.0              1.0            

2. Data Cleaning

3. Data Visualization

In [15]:
import matplotlib.pyplot as plt

4. Hypothesis Development

5. Model Development

In [7]:
# Simple Regression

In [8]:
# Multiple Regression

In [9]:
# Subset Selection

In [10]:
# Regularization

In [11]:
# PCA

In [12]:
# Nonlinear Models

In [13]:
# Model Comparison

In [14]:
# Cross Validation